In [1]:
import torch
import math
import os


if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")

verbose = False

GPU: NVIDIA RTX A6000 is available.


In [2]:
!nvidia-smi

Mon Jan  6 12:50:36 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               Off | 00000000:3B:00.0 Off |                  Off |
| 30%   35C    P8              28W / 300W |     25MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## MeMo Tokenizer and input

In [3]:
from MeMoPyTorch.modelling_memo_tokenizer import MeMoTokenizer

In [4]:
max_length = 12 
tokenizer = MeMoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b", 
                                          truncation_side = 'left',
                                          padding_side='left', max_length=max_length, head_number=4)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPTNeoXTokenizer'. 
The class this function is called from is 'MeMoTokenizer'.


Setting pad token and pad token id = <|endoftext|>, 0


In [5]:
with open("testo_di_prova.txt") as my_first_text_f:
    my_first_text = my_first_text_f.read()

token_ids = tokenizer.encode(my_first_text)#, return_tensors='pt')
print(token_ids) # return max len + 1 

(tensor([[18886,   256, 36144,  4164,  1809,    80,  1448,   295,   532,  1584,
            13, 50190]]), tensor([[  256, 36144,  4164,  1809,    80,  1448,   295,   532,  1584,    13,
         50190,    15]]))


In [6]:
memo_input = tokenizer.get_text_batch_encoding([my_first_text, my_first_text[0:10]])
memo_input.keys()

dict_keys([0, 1, 2, 3])

In [7]:
memo_input[0]['input_ids'].shape

torch.Size([52, 12])

## MeMo Embedding layer

In [8]:
from MeMoPyTorch.modelling_memo_embedding import MeMoEmbedding

In [9]:
d,h,l = 1024, 4, 3

In [10]:
embedding = MeMoEmbedding(
    num_embeddings=tokenizer.vocab_size,
    embedding_dim=d,
    padding_idx=tokenizer.pad_token_id, #0
    _freeze=True
)

MeMo embedding initilialization


In [11]:
input_tokens_ids = tokenizer(['Test', 'Un altro Test'])['input_ids']
print(input_tokens_ids)

input_embeddings = embedding.forward(input_tokens_ids)
input_embeddings

tensor([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
         5089],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0, 2447, 6945,  287,
         6004]])


tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0528,  0.0065, -0.0006,  ...,  0.0034, -0.0194, -0.0465]],

        [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [-0.0435,  0.0142, -0.0113,  ..., -0.0632,  0.0571, -0.0172],
         [ 0.0699,  0.0077,  0.0418,  ...,  0.0119,  0.0123,  0.0719],
         [-0.0145,  0.0034, -0.0103,  ...,  0.0567, -0.0479, -0.0096]]])

In [12]:
memo_input = tokenizer.get_text_batch_encoding([my_first_text, my_first_text[10:30]])

memo_input[0]['input_ids'].shape

torch.Size([52, 12])

In [13]:
memo_input[0]['input_ids'][0:10]

tensor([[    0,     0,     0,     0,     0,     0,     0,     0,     0, 38577,
         17622,  1073],
        [  372,     8,  9718,    74,   843,   936,  4164, 43876, 41380,   258,
           367,   727],
        [ 5507,   313, 15723,   445,  2721,    13,  3435,  3414,   358,  3381,
         15410,    26],
        [ 9776,  1266,    74,    13,   337, 11703,   639, 39337,  1638,  1540,
            10, 12187],
        [  440,  2314,  4173,   299,  8913,  2942,   250,   352,  6770,    80,
            13,  2248],
        [  861,   410,   372, 32924,  1073, 33813,   445,  2721,   299,  2248,
            80,  1484],
        [ 1073,   659,  4611,  1073,   391,   300,   466,  5711, 14804,  1431,
           304, 19702],
        [   74,    15, 14929,  1327,  1323, 10081, 24843, 15438,   412, 16406,
         38055,  9821],
        [ 3737,  1073,   391,   300,   466,  5711, 39814,   260,   770,  5991,
           313,  1962],
        [18006, 22217, 42722, 10863,   262,  7958,  1593, 12704,  5940,  

In [14]:
input_embeddings = embedding.encode(memo_input[0]['input_ids'])
output_symbols = embedding.encode(memo_input[0]['labels'])

input_embeddings.shape, output_symbols.shape

(torch.Size([52, 12, 1024]), torch.Size([52, 12, 1024]))

In [15]:
decoded, _ = embedding.decode(input_embeddings)
print(decoded.shape)

decoded[0:10]

torch.Size([52, 12])


tensor([[    0,     0,     0,     0,     0,     0,     0,     0,     0, 38577,
         17622,  1073],
        [  372,     8,  9718,    74,   843,   936,  4164, 43876, 41380,   258,
           367,   727],
        [ 5507,   313, 15723,   445,  2721,    13,  3435,  3414,   358,  3381,
         15410,    26],
        [ 9776,  1266,    74,    13,   337, 11703,   639, 39337,  1638,  1540,
            10, 12187],
        [  440,  2314,  4173,   299,  8913,  2942,   250,   352,  6770,    80,
            13,  2248],
        [  861,   410,   372, 32924,  1073, 33813,   445,  2721,   299,  2248,
            80,  1484],
        [ 1073,   659,  4611,  1073,   391,   300,   466,  5711, 14804,  1431,
           304, 19702],
        [   74,    15, 14929,  1327,  1323, 10081, 24843, 15438,   412, 16406,
         38055,  9821],
        [ 3737,  1073,   391,   300,   466,  5711, 39814,   260,   770,  5991,
           313,  1962],
        [18006, 22217, 42722, 10863,   262,  7958,  1593, 12704,  5940,  

In [16]:
decoded[0:10] == memo_input[0]['input_ids'][:10]

tensor([[True, True, True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True]])

In [17]:
sims = embedding.weight @ embedding.weight.T
display(sims)
diag_sum = torch.sum(sims[1:, 1: ].diag()) # almost 1 in each entry
print(diag_sum) # obs vs expected
print(torch.sum(sims[1:, 1:]) - diag_sum) #almost 0... more or less

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  9.7387e-01, -2.0831e-02,  ..., -6.7461e-03,
         -1.3701e-02,  7.4025e-02],
        [ 0.0000e+00, -2.0831e-02,  9.0578e-01,  ..., -1.8973e-04,
         -4.8886e-03,  1.4482e-02],
        ...,
        [ 0.0000e+00, -6.7461e-03, -1.8973e-04,  ...,  1.0102e+00,
          5.3430e-03,  2.7948e-02],
        [ 0.0000e+00, -1.3701e-02, -4.8886e-03,  ...,  5.3430e-03,
          1.0213e+00, -2.1110e-02],
        [ 0.0000e+00,  7.4025e-02,  1.4482e-02,  ...,  2.7948e-02,
         -2.1110e-02,  9.3248e-01]])

tensor(50256.9531)
tensor(-1809.0508)


## Test layer and MeMo CMM

In [18]:
from MeMoPyTorch.modelling_memo_layer import MeMoLayer, ProjectionSequence, ProjectionTokens, CorrelationMatrixMemory

### Check initialization of each matrix

In [19]:
d,h,l = 1024, 4, 3
proj = ProjectionSequence(d, d*h)
print(proj.weight.shape, proj.extra_repr())
(proj.weight.T @ proj.weight).diag(), (proj.weight @ proj.weight.T).diag()

torch.Size([4096, 1024]) (trasposed wrt saved one) in_features=4096, out_features=1024


(tensor([1.0111, 1.0151, 0.9992,  ..., 1.0209, 1.0272, 0.9926],
        grad_fn=<DiagonalBackward0_copy>),
 tensor([0.2354, 0.2636, 0.2284,  ..., 0.2459, 0.2440, 0.2607],
        grad_fn=<DiagonalBackward0_copy>))

In [20]:
Prj = torch.normal(0, 1/math.sqrt(d*h), size=(d,d*h))
Prj = torch.transpose(Prj, 0, 1)
(Prj.T @ Prj).diag(), (Prj @ Prj.T).diag()

(tensor([0.9997, 0.9920, 0.9780,  ..., 1.0067, 0.9848, 1.0196]),
 tensor([0.2666, 0.2448, 0.2661,  ..., 0.2471, 0.2549, 0.2301]))

In [21]:
print(d)
print(h)

d_k = d // h

W_v = ProjectionTokens(d, d_k)

print(W_v.weight.shape, W_v.extra_repr())


### always used transposed! so check with .T
(W_v.weight.T @ W_v.weight).diag(), (W_v.weight @ W_v.weight.T).diag()

1024
4
torch.Size([256, 1024]) in_features=1024, out_features=256


(tensor([1.1375, 0.8553, 1.1932,  ..., 0.9809, 0.9940, 1.0911],
        grad_fn=<DiagonalBackward0_copy>),
 tensor([4.0944, 4.3662, 4.1499, 3.9115, 4.0332, 3.7758, 3.8816, 4.0762, 3.7855,
         3.7024, 4.0539, 3.9393, 3.9938, 3.9891, 3.9983, 3.7583, 3.9233, 4.0385,
         3.7158, 4.0627, 4.1810, 4.0674, 3.9983, 4.0403, 4.1708, 3.9498, 4.0763,
         4.1959, 3.9431, 4.0837, 4.3048, 3.9406, 3.9918, 4.0347, 3.9176, 4.0153,
         4.0267, 4.0127, 4.2561, 4.2231, 3.9320, 4.1650, 4.1708, 4.1787, 3.9796,
         4.1406, 4.2367, 4.0490, 3.9934, 4.0441, 4.0678, 3.9038, 3.7231, 4.3075,
         3.8190, 3.8180, 3.9440, 4.2373, 4.1515, 4.4063, 4.1101, 3.9553, 4.1084,
         3.9260, 4.1120, 3.8591, 3.6949, 3.5403, 3.7548, 3.9276, 3.5118, 3.8606,
         3.8915, 4.0128, 3.9460, 3.9313, 3.9781, 4.0359, 4.2151, 3.9924, 3.8414,
         3.8442, 4.0736, 3.8207, 3.8022, 4.1177, 4.1814, 4.0023, 3.9826, 3.9257,
         4.0215, 4.1637, 4.3912, 3.8477, 3.9327, 4.0189, 3.9783, 4.1549, 3.7355,
  

In [22]:
W_v_single_head = torch.normal(0, 1/math.sqrt(d_k), size=(d,d_k))

(W_v_single_head.T @ W_v_single_head).diag(), (W_v_single_head @ W_v_single_head.T).diag()

(tensor([4.1705, 4.1250, 3.8692, 4.1137, 4.1421, 3.7921, 3.8326, 3.9890, 3.7700,
         4.2739, 4.1052, 3.5997, 4.2551, 4.1012, 3.9454, 3.6827, 4.3464, 4.0099,
         3.9253, 4.1895, 3.9990, 3.9708, 3.9141, 3.9289, 3.6128, 3.9473, 3.9000,
         3.9166, 3.8412, 3.7936, 4.0392, 4.1073, 4.1459, 3.9678, 4.2331, 3.7868,
         4.0490, 3.6427, 4.3937, 3.9155, 3.9302, 4.1314, 4.2174, 3.5004, 3.6815,
         3.9610, 4.0613, 3.5549, 4.0337, 4.1838, 3.8771, 3.5905, 3.8546, 4.0931,
         3.8110, 4.0212, 3.8091, 3.9222, 4.0263, 3.6123, 3.9030, 4.0725, 3.9027,
         3.9014, 3.4827, 3.9914, 4.2363, 3.9830, 4.2110, 3.6383, 4.1779, 3.9570,
         3.7747, 3.9732, 3.7706, 4.2763, 4.1890, 4.1636, 4.2126, 3.9425, 3.8754,
         4.1642, 4.0893, 3.7432, 4.3139, 3.8557, 3.9961, 3.7226, 3.8980, 3.9085,
         4.0172, 4.0753, 4.0018, 4.2037, 3.9887, 3.8457, 4.0470, 4.1462, 3.9458,
         4.0999, 4.0249, 3.9487, 3.9969, 3.9923, 3.8076, 4.2088, 3.9903, 3.9782,
         4.0395, 4.0783, 4.0

### Check memorization on single layer

In [23]:
d,h,l = 1024, 4, 3

layer = MeMoLayer(d, h)
layer

MeMoLayer(
  (W_v_single_head): ProjectionTokens(in_features=1024, out_features=256)
  (Prj): ProjectionSequence((trasposed wrt saved one) in_features=4096, out_features=1024)
  (CMM): CorrelationMatrixMemory(in_features=1024, out_features=1024)
)

In [24]:
batch_size, current_length, d = input_embeddings.shape
batch_size, current_length, d 

(52, 12, 1024)

In [25]:
output_symbols.shape

torch.Size([52, 12, 1024])

In [26]:
current_length = int(input_embeddings.shape[1]/ h)

input_sequence = input_embeddings.reshape((batch_size, current_length, h, d))

current_output_symbols = output_symbols[:, [(x+1)*h-1 for x in range(0,current_length)]]
j = 2 
print(sum(sum(input_sequence[0][j] == input_embeddings[0][4*j:4*(j+1)])), input_sequence[0][j].shape)

(batch_size, blocks,h,d) = input_sequence.shape

tensor(4096) torch.Size([4, 1024])


In [27]:
input_sequence.shape, current_output_symbols.shape

(torch.Size([52, 3, 4, 1024]), torch.Size([52, 3, 1024]))

In [28]:
layer = MeMoLayer(d, h)
display(layer)
## update the input sequence for the next layer
_, seq_encoding_for_the_last_layer = layer.memorize(input_sequence, current_output_symbols, is_last=False)
layer.directly_memorize(seq_encoding_for_the_last_layer)

MeMoLayer(
  (W_v_single_head): ProjectionTokens(in_features=1024, out_features=256)
  (Prj): ProjectionSequence((trasposed wrt saved one) in_features=4096, out_features=1024)
  (CMM): CorrelationMatrixMemory(in_features=1024, out_features=1024)
)

In [29]:
input_sequence.shape, input_sequence[3].shape # batch (52 elements of chunks 4*4*1024)

(torch.Size([52, 3, 4, 1024]), torch.Size([3, 4, 1024]))

In [30]:
_, seq_encoding_for_the_last_layer = layer.retrieve(input_sequence)

print(seq_encoding_for_the_last_layer.shape)

torch.Size([52, 1024])


In [31]:
logits = layer.directly_retrieve(seq_encoding_for_the_last_layer)

In [32]:
retreived_output_symbol_vector, m = embedding.decode(logits)
print(retreived_output_symbol_vector, m)

tensor([48505, 20110,  1108, 48019,    80, 19216,  9718,  1113,  4927,    66,
        19216,  1448,  2122, 41530,   187,  4172,   246,   659, 10986, 30975,
           80, 12931,   352, 14134,  2721,   258,  8830,    87,   826,    15,
        17532,   729, 26798, 41070,  6575,   299,   266,  3737, 20889,   287,
          512,   354,   250,   247,    70,   275, 16128,  2680,    74, 13679,
           15,   843]) tensor([0.5730, 1.0270, 0.9928, 1.1156, 0.9149, 1.4244, 1.0947, 1.0112, 1.0783,
        1.8724, 1.4561, 1.0143, 1.0809, 0.8852, 0.8631, 1.0775, 1.0035, 0.7843,
        1.0244, 0.9231, 0.9616, 1.1672, 0.9987, 0.9383, 1.2387, 0.9683, 1.0902,
        0.9808, 0.9863, 1.1337, 1.0783, 0.9966, 0.9620, 0.9934, 1.1199, 0.9827,
        0.9565, 0.8484, 0.9535, 1.0549, 1.1408, 1.0350, 1.0756, 0.9516, 0.8990,
        1.0362, 0.8466, 1.2380, 1.0878, 0.9070, 1.0193, 1.1173],
       grad_fn=<MaxBackward0>)


In [33]:
o = embedding.decode(current_output_symbols[:, -1])[0]
display(o)

print(sum(o == retreived_output_symbol_vector), 'over', retreived_output_symbol_vector.shape)

tensor([48505, 20110,  1108, 48019,    80, 19216,  9718,  1113,  4927,    66,
        19216,  1448,  2122, 41530,   187,  4172,   246,   659, 10986, 30975,
           80, 12931,   352, 14134,  2721,   258,  8830,    87,   826,    15,
        17532,   729, 26798, 41070,  6575,   299,   266,  3737, 20889,   287,
          512,   354,   250,   247,    70,   275, 16128,  2680,    74, 13679,
           15,   209])

tensor(51) over torch.Size([52])


In [34]:
#### test single block

In [35]:
print(input_sequence[3].shape, current_output_symbols[3].shape)
print(input_sequence[3][0].shape, current_output_symbols[3][0].shape)

torch.Size([3, 4, 1024]) torch.Size([3, 1024])
torch.Size([4, 1024]) torch.Size([1024])


In [36]:
total = 0
correct = 0 

for batch_index in range(len(memo_input[0]['input_ids'])):
    #print("input ids", memo_input[0]['input_ids'][batch_index])
    #print()
    
    for i in range(len(current_output_symbols[batch_index])):
        #display(embedding.decode(input_sequence[batch_index][i]), embedding.decode(current_output_symbols[batch_index][i]))
        true = embedding.decode(current_output_symbols[batch_index][i])[0].item()
        
        _, seq_encoding_for_the_last_layer  = layer.retrieve(input_sequence[batch_index][i].unsqueeze(0).unsqueeze(0))
        
        retreived_output_symbol_vector, m = embedding.decode(layer.directly_retrieve(seq_encoding_for_the_last_layer))
        pred = retreived_output_symbol_vector.item()

        total += 1
        correct += pred == true

print(f"{correct}/{total}")

153/156


### Check with batch size of 1 and output probs

In [37]:
memo_input = tokenizer.get_text_batch_encoding(['this is a test for a very short short sequence of 12 tokens'])[0]
input_ids, labels = memo_input['input_ids'], memo_input['labels']
print(input_ids, labels)

input_embeddings = embedding.encode(input_ids)
#print(input_embeddings.shape)

output_embeddings = embedding.encode(labels)
#print(output_embeddings.shape)


current_length = max_length

current_length = int(current_length/h)
input_sequence = input_embeddings.reshape((1, current_length, h, d))

output_symbols = output_embeddings[:, [(x+1)*h-1 for x in range(0,current_length)]] ## the output symbol is always the same tokem?
print(embedding.decode(output_symbols))
input_sequence.shape, output_symbols.shape

tensor([[2520,  310,  247, 1071,  323,  247, 1077, 2159, 2159, 3425,  273, 1249]]) tensor([[  310,   247,  1071,   323,   247,  1077,  2159,  2159,  3425,   273,
          1249, 21761]])
(tensor([[  323,  2159, 21761]]), tensor([[0.9743, 1.0260, 1.0412]]))


(torch.Size([1, 3, 4, 1024]), torch.Size([1, 3, 1024]))

In [38]:
embedding.decode(input_sequence)[0], embedding.decode(output_symbols)[0], input_ids

(tensor([[[2520,  310,  247, 1071],
          [ 323,  247, 1077, 2159],
          [2159, 3425,  273, 1249]]]),
 tensor([[  323,  2159, 21761]]),
 tensor([[2520,  310,  247, 1071,  323,  247, 1077, 2159, 2159, 3425,  273, 1249]]))

In [39]:
input_sequence.shape

torch.Size([1, 3, 4, 1024])

In [40]:
output_symbols.shape

torch.Size([1, 3, 1024])

In [41]:
layer = MeMoLayer(d, h)
display(layer)

## update the input sequence for the next layer
_, seq_encoding_for_the_last_layer = layer.memorize(input_sequence, output_symbols, is_last=False)
layer.directly_memorize(seq_encoding_for_the_last_layer)

for i in range(0,3):
    _, seq_encoding_for_the_last_layer = layer.retrieve(input_sequence[0][i].unsqueeze(0).unsqueeze(0))
    print(seq_encoding_for_the_last_layer.shape)
                                                        
    retreived_output_symbol_vector, m = embedding.decode(layer.directly_retrieve(seq_encoding_for_the_last_layer))
    print(retreived_output_symbol_vector, m)

MeMoLayer(
  (W_v_single_head): ProjectionTokens(in_features=1024, out_features=256)
  (Prj): ProjectionSequence((trasposed wrt saved one) in_features=4096, out_features=1024)
  (CMM): CorrelationMatrixMemory(in_features=1024, out_features=1024)
)

torch.Size([1, 1024])
tensor([323]) tensor([0.9204], grad_fn=<MaxBackward0>)
torch.Size([1, 1024])
tensor([2159]) tensor([0.9565], grad_fn=<MaxBackward0>)
torch.Size([1, 1024])
tensor([21761]) tensor([0.9486], grad_fn=<MaxBackward0>)


## Test the entire MeMo model

In [42]:
from MeMoPyTorch.modelling_memo import MeMo

In [43]:
from MeMoPyTorch.modelling_memo_tokenizer import MeMoTokenizer

In [44]:
with open("testo_di_prova.txt") as my_first_text_f:
    my_first_text = my_first_text_f.read()

In [45]:
max_length = 384
print(max_length, h)
tokenizer = MeMoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b", 
                                          padding_side='left', truncation_side='left', 
                                          max_length=max_length, head_number=h)

384 4


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPTNeoXTokenizer'. 
The class this function is called from is 'MeMoTokenizer'.


Setting pad token and pad token id = <|endoftext|>, 0


In [46]:
memo_input = tokenizer.memo_heads_encode(my_first_text[0:10])
memo_input[0]['labels'].shape

torch.Size([1, 384])

In [47]:
device='cuda:0' #'cpu'

In [55]:
d,h,l

(1024, 4, 3)

In [56]:
model = MeMo(inner_dim=d, 
             num_of_heads=h, 
             num_of_layers=3, 
             chunk_length=max_length, 
             num_embeddings=tokenizer.vocab_size, 
             padding_idx=tokenizer.pad_token_id, 
             device=device) #MeMoModel
model

MeMo embedding initilialization


MeMo(
  (encoder): MeMoEmbedding(50254, 1024, padding_idx=0)
  (layers): ModuleList(
    (0-2): 3 x MeMoLayer(
      (W_v_single_head): ProjectionTokens(in_features=1024, out_features=256)
      (Prj): ProjectionSequence((trasposed wrt saved one) in_features=4096, out_features=1024)
      (CMM): CorrelationMatrixMemory(in_features=1024, out_features=1024)
    )
  )
)

In [57]:
import tqdm


class Evaluation:
    def check_memorization(self, model, tokenizer, text, # device='cpu',
                           starting_point=None):
        if starting_point == None:
            basic_block = model.h ** model.l
        else:
            basic_block = starting_point
        
        
        input_ = tokenizer(my_first_text, padding='longest', truncation='do_not_truncate', max_length=None)
        input_ = tokenizer.pad(input_, pad_to_multiple_of=basic_block)
        input_ids = input_['input_ids']
                
        count = 0
        correct = 0
        max_length = tokenizer.max_length
        (batch_size, number_of_tokens) = input_ids.shape

        #print(f"(batch_size, number_of_tokens) = {(batch_size, number_of_tokens)}")
        
        for i in tqdm.tqdm(range(basic_block,  number_of_tokens - 1)):
            text_tokens = input_ids[:, i - basic_block:i]
            
            (batch_size, number_of_tokens) = text_tokens.shape
            
            text_tokens = torch.concat((torch.zeros((batch_size, max_length-1-number_of_tokens), 
                                                    dtype=torch.int), 
                                        text_tokens), axis=1
                                      )
            
            #print(i - basic_block, i)
            out, max_value = model.retrieve(text_tokens)
            #print(out, input_ids[:, i])
            #print(out[0].item())
            
            count += batch_size
            correct += torch.sum(out.to('cpu') == input_ids[:, i])
        
                           
        return correct / count

    def check_pretokenized(self, model, tokenizer, input_ids,# device='cpu',
                           starting_point=None):
        if starting_point == None:
            basic_block = model.h ** model.l
        else:
            basic_block = starting_point
                
        count = 0
        correct = 0
        max_length = tokenizer.max_length
        (batch_size, number_of_tokens) = input_ids.shape

        #print(f"(batch_size, number_of_tokens) = {(batch_size, number_of_tokens)}")
        
        for i in tqdm.tqdm(range(basic_block,  number_of_tokens - 1)):
            text_tokens = input_ids[:, i - basic_block:i]
            
            (batch_size, number_of_tokens) = text_tokens.shape
            
            text_tokens = torch.concat((torch.zeros((batch_size, max_length-1-number_of_tokens), 
                                                    dtype=torch.int), 
                                        text_tokens), axis=1
                                      )
            
            #print(i - basic_block, i)
            out, max_value = model.retrieve(text_tokens)
            #print(out, input_ids[:, i])
            #print(out[0].item())
            
            count += batch_size
            correct += torch.sum(out.to('cpu') == input_ids[:, i])
        
                           
        return correct / count
        

In [65]:
model = MeMo(inner_dim=d, 
             num_of_heads=h, 
             num_of_layers=3, 
             chunk_length=max_length, 
             num_embeddings=tokenizer.vocab_size, 
             padding_idx=tokenizer.pad_token_id, 
             device=device)

memo_input = tokenizer.get_text_batch_encoding([my_first_text]*8)
model.memorize_text(memo_input)

e = Evaluation()
out = e.check_pretokenized(model, tokenizer, memo_input[0]['input_ids'])
print("Degree of memorization: %f ", out)

MeMo embedding initilialization


100%|██████████| 319/319 [00:00<00:00, 743.33it/s]

Degree of memorization: %f  tensor(0.1379)


In [66]:
model = MeMo(inner_dim=d, 
             num_of_heads=h, 
             num_of_layers=3, 
             chunk_length=max_length, 
             num_embeddings=tokenizer.vocab_size, 
             padding_idx=tokenizer.pad_token_id, 
             device=device)
print("CMM pre learning")
display(model.layers[0].CMM.weight)



memo_input = tokenizer.get_text_batch_encoding([my_first_text]*8)
(bs, ml) = memo_input[0]['input_ids'].shape

for b in range(bs):
    memo_single_input = {h: {k: memo_input[h][k][b].unsqueeze(0) for k in memo_input[h]} for h in memo_input}
    #print(memo_single_input[0]['input_ids'].shape)

    model.memorize_text(memo_single_input)

e = Evaluation()
out = e.check_pretokenized(model, tokenizer, memo_input[0]['input_ids'])
print("Degree of memorization: %f ", out)

MeMo embedding initilialization
CMM pre learning


Parameter containing:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0', requires_grad=True)

100%|██████████| 319/319 [00:00<00:00, 743.65it/s]

Degree of memorization: %f  tensor(0.9122)


In [67]:
Prj = model.layers[0].Prj.weight.detach().cpu()
CMM = model.layers[0].CMM.weight.detach().cpu()

Prj.T @ Prj

tensor([[ 9.8602e-01, -1.5501e-02,  1.4697e-02,  ..., -4.7693e-03,
          1.7892e-03, -9.1244e-03],
        [-1.5501e-02,  1.0178e+00, -1.2891e-03,  ...,  3.1778e-04,
          5.4323e-04,  2.4665e-02],
        [ 1.4697e-02, -1.2891e-03,  9.5536e-01,  ...,  3.3318e-02,
          4.7871e-03,  3.0963e-02],
        ...,
        [-4.7693e-03,  3.1778e-04,  3.3318e-02,  ...,  9.9556e-01,
          2.4344e-02,  1.1201e-02],
        [ 1.7892e-03,  5.4323e-04,  4.7871e-03,  ...,  2.4344e-02,
          1.0154e+00,  4.7761e-03],
        [-9.1244e-03,  2.4665e-02,  3.0963e-02,  ...,  1.1201e-02,
          4.7761e-03,  1.0402e+00]])

In [68]:
CMM

tensor([[-0.0176, -0.0157, -0.0027,  ...,  0.0195, -0.0135, -0.0177],
        [-0.0045,  0.0303, -0.0263,  ..., -0.0104,  0.0133,  0.0261],
        [-0.0107, -0.0095,  0.0075,  ..., -0.0242,  0.0158,  0.0182],
        ...,
        [ 0.0079,  0.0222,  0.0141,  ...,  0.0181,  0.0083, -0.0247],
        [ 0.0100,  0.0099,  0.0003,  ..., -0.0412,  0.0112, -0.0177],
        [-0.0092, -0.0246, -0.0309,  ..., -0.0188,  0.0095,  0.0073]])

In [69]:
out = e.check_pretokenized(model, tokenizer, memo_input[0]['input_ids'])
print("Degree of memorization: %f ", out)

100%|██████████| 319/319 [00:00<00:00, 600.94it/s]

Degree of memorization: %f  tensor(0.9122)


In [70]:
model.forget_text(memo_input)

In [71]:
out = e.check_pretokenized(model, tokenizer, memo_input[0]['input_ids'])
print("Degree of memorization: %f ", out)

100%|██████████| 319/319 [00:00<00:00, 604.12it/s]

Degree of memorization: %f  tensor(0.0940)
